## 1. Progress Report (Student’s Voice)

- **DICOM CT volume I/O with SimpleITK**  
  I implemented `load_ct_volume` to read entire CT series from a folder using `sitk.ImageSeriesReader`. The function applies any Rescale Slope/Intercept metadata automatically, and prints out volume shape, spacing, slope, and intercept (e.g., `shape=(207,512,512), spacing=(0.78125,0.78125,2.5), slope=1.0, intercept=0.0` for the reference CT). 

- **Segmentation import & frame-to-slice mapping**  
  I wrote `load_segmentation` to read a DICOM-RTSEG with SimpleITK, then used pydicom to extract `PerFrameFunctionalGroupsSequence` and map each frame’s physical Z-position to the corresponding CT slice index. It loaded 15 frames into a full 3D mask, printing each frame’s `z_phys` and `slice_index`. 

- **Dual‐CT dataset handling**  
  The notebook loads two CT series—“10_AP_Ax2.50mm” as the reference and “30_EQP_Ax2.50mm” as the input—producing `ref_vol.shape=(207,512,512)` and `inp_vol.shape=(111,512,512)`. This sets up atlas‐based propagation of the segmentation. 

- **Bounding‐box and centroid computation**  
  Using NumPy’s `argwhere`, I computed the bounding box (`Z:[152,183] Y:[198,336] X:[72,306]`) and centroid index `(176,229,117)` of the reference mask to localize the ROI. 

- **Seed‐point transform to input CT**  
  I transformed the reference centroid’s physical coordinates `(-121.59375, 69.90625, -57.5)` into the input CT index space, yielding a seed index of `(z,y,x)=(71,229,127)` for region growing. 

- **3D region‐growing segmentation**  
  Defined `RegionGrowing3D` to flood‐fill voxels within intensity bounds (−50 to 150 HU) starting from the seed, restricted to a padded bounding box (±5 voxels). Produced a raw mask (`auto_raw`). 

- **Post‐processing of raw mask**  
  Applied `binary_fill_holes` and labeled connected components, removing all components smaller than 500 voxels. (A warning about overflow in voxel‐count subtraction indicates I need to fix the subtraction logic.) 

- **Quantitative evaluation metrics**  
  Resampled the ground-truth segmentation onto the input CT grid with nearest‐neighbor interpolation, then computed Dice (0.0015), Jaccard (0.0010), and volume difference (7032.59 cm³) using `EvalMetrics`. These very low overlap scores and huge volume discrepancy highlight that the region growing parameters or seed placement need refinement. 

- **Slice‐by‐slice overlay visualization**  
  Overlaid GT contours in green and auto‐segmentation contours in red on three slices around the volume midpoint (mid ± 10). The overlays clearly show almost no overlap, underscoring the need for improved segmentation strategy. 

---

## 2. Student’s Open Questions

1. **Rescale slope/intercept reliability**  
   I hard-coded reading of DICOM tags `0028|1053` and `0028|1052`. How should I handle cases where slope/intercept are missing or stored differently?

2. **Frame-to-slice mapping edge cases**  
   My mapping relies on exact Z-position matching with `np.isclose`. What tolerance should I use, and how to handle cases where no slice matches due to rounding or different coordinate origins?

3. **Overflow in post-processing subtraction**  
   I saw `RuntimeWarning: overflow encountered in scalar subtract` when computing removed voxels. How do I correctly compute and log the number of voxels removed without overflow?

4. **Region growing within ROI bounding box**  
   I passed an `roi` tuple to `RegionGrowing3D` but never used it in `segment()`. Should I crop the volume to the ROI first or add boundary checks to limit the search?

5. **Parameter selection for region growing**  
   Intensity bounds (−50 to 150 HU) were chosen arbitrarily. What’s a recommended workflow to tune these thresholds (e.g., histogram analysis, interactive tools)?

6. **Resampling GT to input grid**  
   Is nearest-neighbor interpolation sufficient for binary masks, or should I use distance‐based or smoother interpolators to better align geometries?

7. **Interpreting low Dice/Jaccard**  
   I got Dice = 0.0015 and volume difference > 7000 cm³. Does this always imply a bad segmentation, or could coordinate misalignment be the culprit? How to isolate the source of error?

---

## 3. Cell-by-Cell Logical Explanation

### **Imports & Auto-Reload**
```python
%load_ext autoreload
%autoreload 2

import os, numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from scipy.ndimage import binary_fill_holes, label
from scipy.spatial.distance import directed_hausdorff
```
- **What:** Loads core libraries for I/O, array ops, image processing, plotting, morphology, and distance metrics.  
- **Why:** Ensures utility modules can be edited on the fly and all required packages are imported.  
- **Workflow:** Sets up the environment for all subsequent data handling and analysis. 

---

### **Funzioni di I/O DICOM**
```python
def load_ct_volume(folder_path):
    …  # Uses sitk.ImageSeriesReader, applies slope/intercept
    return img, arr

def load_segmentation(dcm_path, ref_img):
    …  # Reads RTSEG, maps frames to CT slices via pydicom
    return seg, full_mask
```
- **What:** Defines two functions: one to load CT volumes (applying rescale), another to load and spatially align segmentation frames to the CT.  
- **Why:** Abstracts away DICOM parsing and mapping logic so downstream code can simply call these utilities.  
- **Workflow:** Provides `ref_ct`, `ref_vol`, and `ref_mask_full` for the rest of the pipeline. 

---

### **Load dataset**
```python
ref_ct, ref_vol = load_ct_volume("../dataset/10_AP_Ax2.50mm")
inp_ct, inp_vol = load_ct_volume("../dataset/30_EQP_Ax2.50mm")
seg_img, ref_mask_full = load_segmentation(
    "../dataset/10_AP_Ax2.50mm_ManualROI_Tumor.dcm", ref_ct
)
```
- **What:** Loads the reference CT, input CT, and reference tumor mask; prints shapes, spacings, and frame mappings.  
- **Why:** Prepares both imaging volumes and the ground truth mask for segmentation propagation.  
- **Workflow:** Establishes core data objects used in bounding‐box computation and region growing. 

---

### **Calcolo bounding-box e centroide su reference mask**
```python
coords = np.argwhere(ref_mask_full)
zmin, … = coords.min(axis=0)
zmax, … = coords.max(axis=0)
centroid = coords.mean(axis=0).astype(int)
```
- **What:** Computes min/max indices along each axis to define the ROI bounding box, and calculates its centroid in index space.  
- **Why:** Localizes the region of interest to reduce computation and pick a robust seed point.  
- **Workflow:** Supplies `zb, yb, xb` and `centroid` for subsequent seed‐point mapping and region growing. 

---

### **Seed transform**
```python
world = ref_ct.TransformIndexToPhysicalPoint(idx)
seed_idx = inp_ct.TransformPhysicalPointToIndex(world)
```
- **What:** Converts the centroid index from the reference CT into physical coordinates, then back to index coordinates on the input CT.  
- **Why:** Ensures the seed used for region growing lands in the same anatomical location in the new volume.  
- **Workflow:** Outputs `seed_idx` for `RegionGrowing3D.segment()`. 

---

### **Region Growing nel Bounding Box**
```python
class RegionGrowing3D:
    def segment(self, seed):
        …  # stack-based flood fill within intensity range
```
- **What:** Implements a simple 6-connected flood fill, adding neighbors if they fall within HU bounds.  
- **Why:** Automatically extracts a contiguous region around the seed that matches expected tissue intensities.  
- **Workflow:** Produces `auto_raw`, the unfiltered segmentation candidate for the input CT. 

---

### **Post-processing**
```python
def clean_mask(mask, min_vox=500):
    fill = binary_fill_holes(mask)
    labs, n = label(fill)
    …  # remove small components
auto_mask = clean_mask(auto_raw)
```
- **What:** Fills holes in the raw mask, labels connected components, and removes any with fewer than 500 voxels.  
- **Why:** Cleans spurious islands and ensures the segmentation is a single, cohesive object.  
- **Workflow:** Outputs `auto_mask`, the final binary prediction used for evaluation. 

---

### **Classe per le metriche**
```python
# Resample GT mask with sitk.ResampleImageFilter
# EvalMetrics.dice, jaccard, vol_diff
dice_v = EvalMetrics.dice(ref_mask_on_input, auto_mask)
…
print(f"Dice: {dice_v:.4f}, …")
```
- **What:** Resamples the ground-truth mask onto the input CT grid, then computes Dice, Jaccard, and volume difference in cm³.  
- **Why:** Provides quantitative measures of segmentation quality.  
- **Workflow:** Outputs evaluation metrics highlighting the (very low) overlap between GT and auto masks. 

---

### **Visualizzazione slice con overlay**
```python
def overlay_slice(ct, gt, auto, z):
    …  # imshow + contour(gt, green) + contour(auto, red)
for dz in [-10,0,10]:
    overlay_slice(inp_vol, ref_mask_on_input, auto_mask, z)
```
- **What:** Displays three axial slices centered around the volume midpoint with GT and auto contours overlaid.  
- **Why:** Offers visual confirmation of segmentation performance (or failure).  
- **Workflow:** Concludes the notebook by visually showing the gap between the predicted region and the ground truth. 